## Data Ingestion

In [1]:
import pandas as pd

# column_names = ['id', 'text', 'label']
constraint_train = pd.read_csv("/kaggle/input/dataset-4/constraint_train.csv")
constraint_test = pd.read_csv("/kaggle/input/dataset-4/constraint_test.csv")
constraint_dev = pd.read_csv("/kaggle/input/dataset-4/constraint_dev.csv")

In [2]:
constraint_train.shape, constraint_test.shape, constraint_dev.shape

((4494, 4), (963, 4), (963, 4))

## Data Preparation

In [4]:
LABELS = constraint_train['label'].unique().tolist()
train_label = constraint_train['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/1124859992.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_label = constraint_train['label'].replace(LABELS, [0, 1]).tolist()


In [5]:
train_data = constraint_train['tweet'].tolist()

In [6]:
dev_data = constraint_dev['tweet'].tolist()
dev_label = constraint_dev['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/2472027419.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_label = constraint_dev['label'].replace(LABELS, [0, 1]).tolist()


In [7]:
test_data = constraint_test['tweet'].tolist()
test_label = constraint_test['label'].replace(LABELS, [0, 1]).tolist()

/tmp/ipykernel_36/1887347934.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_label = constraint_test['label'].replace(LABELS, [0, 1]).tolist()


In [8]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClassificationDataset(Dataset):
    def __init__(self, texts: list[str], labels: list[int], tokenizer, max_length: int):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens=True, 
            max_length=self.max_length, 
            return_token_type_ids=False, 
            padding="max_length",
            truncation=True, 
            return_attention_mask=True, 
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [9]:
from transformers import BertTokenizer, BertModel, AdamW

max_length = 128

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = ClassificationDataset(train_data, train_label, tokenizer, max_length)
dev_dataset = ClassificationDataset(dev_data, dev_label, tokenizer, max_length)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

In [11]:
test_dataset = ClassificationDataset(test_data, test_label, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Model Preparation

In [12]:
num_classes = 2

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_classes, dropout=0.1):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.ffn(x)
        return logits

In [14]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [15]:
model = BERTClassifier(bert_model, num_classes)

In [16]:
learning_rate = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from tqdm import tqdm

def calculate_metrics(y_true, y_pred, y_pred_proba, num_classes):
    if num_classes == 2:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred),
            'recall': recall_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred)
        }
    else:
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, average='weighted'),
            'recall': recall_score(y_true, y_pred, average='weighted'),
            'f1': f1_score(y_true, y_pred, average='weighted')
        }
    
    # Calculate ROC-AUC score
    if num_classes == 2:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
    else:
        try:
            metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average='weighted')
        except:
            metrics['roc_auc'] = None
            
    return metrics

In [18]:
def evaluate(model, data_loader, device, num_classes):
    model.eval()
    all_labels = []
    all_predictions = []
    all_predictions_proba = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_predictions_proba.extend(probabilities.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)
    all_predictions_proba = np.array(all_predictions_proba)

    return calculate_metrics(all_labels, all_predictions, all_predictions_proba, num_classes)

In [19]:
def train(model, train_loader, val_loader, criterion, optimizer, device, num_epochs):
    best_val_metrics = {'f1': 0.0}
    history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        all_train_labels = []
        all_train_predictions = []
        all_train_predictions_proba = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            probabilities = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_train_labels.extend(labels.cpu().numpy())
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_predictions_proba.extend(probabilities.cpu().detach().numpy())

        # Calculate training metrics
        all_train_labels = np.array(all_train_labels)
        all_train_predictions = np.array(all_train_predictions)
        all_train_predictions_proba = np.array(all_train_predictions_proba)
        train_metrics = calculate_metrics(all_train_labels, all_train_predictions, 
                                       all_train_predictions_proba, num_classes)
        
        # Validation phase
        val_metrics = evaluate(model, val_loader, device, num_classes)
        
        # Store metrics history
        history['train'].append({
            'loss': train_loss / len(train_loader),
            **train_metrics
        })
        history['val'].append(val_metrics)

        # Print epoch results
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("Training Metrics:")
        print(f"Loss: {train_loss/len(train_loader):.4f}")
        for metric, value in train_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
        
        print("\nValidation Metrics:")
        for metric, value in val_metrics.items():
            print(f"{metric.capitalize()}: {value:.4f}")
    
    return history

In [20]:
num_epochs = 5

optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
history = train(model, train_loader, dev_loader, criterion, optimizer, device, num_epochs)

Epoch 1/5: 100%|██████████| 141/141 [00:56<00:00,  2.48it/s]



Epoch 1/5
Training Metrics:
Loss: 0.2743
Accuracy: 0.9072
Precision: 0.9114
Recall: 0.8942
F1: 0.9027
Roc_auc: 0.9601

Validation Metrics:
Accuracy: 0.9637
Precision: 0.9610
Recall: 0.9588
F1: 0.9599
Roc_auc: 0.9926


Epoch 2/5: 100%|██████████| 141/141 [00:56<00:00,  2.51it/s]



Epoch 2/5
Training Metrics:
Loss: 0.0846
Accuracy: 0.9755
Precision: 0.9790
Recall: 0.9700
F1: 0.9745
Roc_auc: 0.9940

Validation Metrics:
Accuracy: 0.9595
Precision: 0.9364
Recall: 0.9771
F1: 0.9563
Roc_auc: 0.9952


Epoch 3/5: 100%|██████████| 141/141 [00:56<00:00,  2.49it/s]



Epoch 3/5
Training Metrics:
Loss: 0.0353
Accuracy: 0.9900
Precision: 0.9903
Recall: 0.9889
F1: 0.9896
Roc_auc: 0.9984

Validation Metrics:
Accuracy: 0.9740
Precision: 0.9813
Recall: 0.9611
F1: 0.9711
Roc_auc: 0.9919


Epoch 4/5: 100%|██████████| 141/141 [00:56<00:00,  2.49it/s]



Epoch 4/5
Training Metrics:
Loss: 0.0273
Accuracy: 0.9915
Precision: 0.9930
Recall: 0.9894
F1: 0.9912
Roc_auc: 0.9989

Validation Metrics:
Accuracy: 0.9637
Precision: 0.9427
Recall: 0.9794
F1: 0.9607
Roc_auc: 0.9946


Epoch 5/5: 100%|██████████| 141/141 [00:56<00:00,  2.49it/s]



Epoch 5/5
Training Metrics:
Loss: 0.0251
Accuracy: 0.9920
Precision: 0.9912
Recall: 0.9921
F1: 0.9917
Roc_auc: 0.9996

Validation Metrics:
Accuracy: 0.9564
Precision: 0.9925
Recall: 0.9108
F1: 0.9499
Roc_auc: 0.9921


In [22]:
import pandas as pd

pd.DataFrame(history['train'])

,loss,accuracy,precision,recall,f1,roc_auc
0,0.274277,0.907210,0.911446,0.894177,0.902729,0.960133
1,0.084637,0.975523,0.979011,0.969963,0.974466,0.993954
2,0.035306,0.989987,0.990282,0.988909,0.989595,0.998400
3,0.027290,0.991544,0.993043,0.989372,0.991204,0.998928
4,0.025055,0.991989,0.991228,0.992144,0.991686,0.999574


In [23]:
pd.DataFrame(history['val'])

,accuracy,precision,recall,f1,roc_auc
0,0.963655,0.961009,0.958810,0.959908,0.992583
1,0.959502,0.936404,0.977117,0.956327,0.995215
2,0.974039,0.981308,0.961098,0.971098,0.991943
3,0.963655,0.942731,0.979405,0.960718,0.994597
4,0.956386,0.992519,0.910755,0.949881,0.992052


In [24]:
evaluate(model, test_loader, device, num_classes)

{'accuracy': 0.9511941848390446,
 'precision': 0.992822966507177,
 'recall': 0.9041394335511983,
 'f1': 0.9464082098061575,
 'roc_auc': 0.9938271604938271}